# Importing Libraries

In [ ]:
import torch.nn as nn
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import random_split
import torch.optim as optim
from torch.utils.data import DataLoader

# Downloading Datasets

In [ ]:
train_dataset = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor(),
    target_transform= Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0,torch.tensor(y), value=1))

)

test_dataset = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor(),
    target_transform= Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0,torch.tensor(y), value=1))

)

100%|██████████| 26421880/26421880 [00:02<00:00, 10037509.94it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 268133.63it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4959706.31it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 22586063.80it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



# Splitting Dataset for Training, Testing and Validating

In [ ]:
train_size = len(train_dataset)
val_size = int(0.1 * train_size)

train_dataset, val_dataset = random_split(train_dataset, [train_size - val_size, val_size])

In [ ]:
print(f"Test set size: {len(test_dataset)}")
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Test set size: 10000
Training set size: 54000
Validation set size: 6000


# Dataloader

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)             # batch size is 64
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Building a Simple Neural Network

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear1 = nn.Linear(in_features=28*28, out_features=256)
        self.linear2 = nn.Linear(in_features=256, out_features=128)
        self.linear3 = nn.Linear(in_features=128, out_features=64)
        self.linear4 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x): # forward prop function
        x = x.view(x.size(0),-1)
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        x = torch.relu(self.linear3(x))
        x = torch.softmax(self.linear4(x),dim = 1)
        return x

simple_net = SimpleNet()

# Variable initialisation for Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(simple_net.parameters(), lr=0.01)

# Training

In [ ]:
def trainval_loop(train_loader, val_loader, simple_net, criterion, optimizer):

    simple_net.train()


    size = len(train_loader.dataset)
    for batch, (X, y) in enumerate(train_loader):

        pred = simple_net(X)
        loss = criterion(pred, y)

                                                 # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0:
            current = batch * len(X)
            print(f"Training loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]") #printing training loss


    simple_net.eval()

    # Validation loop
    val_loss = 0
    val_size = len(val_loader.dataset)
    with torch.no_grad():
        for batch, (X, y) in enumerate(val_loader):
            # Compute prediction and loss
            pred = simple_net(X)
            loss = criterion(pred, y)
            val_loss += loss.item()

            if batch % 100 == 0:
                current = batch * len(X)
                print(f"Validation loss: {loss.item():>7f}  [{current:>5d}/{val_size:>5d}]") #printing validation loss

    avg_val_loss = val_loss / len(val_loader)
    print(f"Avg. Validation loss: {avg_val_loss:>7f}")

In [ ]:
epochs = 10
for t in range(epochs):
    print(f'===EPOCH===  {t}')
    trainval_loop(train_loader,val_loader, simple_net, criterion, optimizer)

===EPOCH===  0
Training loss: 2.300442  [    0/54000]
Training loss: 2.179726  [ 6400/54000]
Training loss: 2.398649  [12800/54000]
Training loss: 2.336150  [19200/54000]
Training loss: 2.304900  [25600/54000]
Training loss: 2.367400  [32000/54000]
Training loss: 2.383025  [38400/54000]
Training loss: 2.383025  [44800/54000]
Training loss: 2.398650  [51200/54000]
Validation loss: 2.320525  [    0/ 6000]
Avg. Validation loss: 2.360363
===EPOCH===  1
Training loss: 2.320525  [    0/54000]
Training loss: 2.398650  [ 6400/54000]
Training loss: 2.336150  [12800/54000]
Training loss: 2.336150  [19200/54000]
Training loss: 2.351775  [25600/54000]
Training loss: 2.367400  [32000/54000]
Training loss: 2.336150  [38400/54000]
Training loss: 2.414275  [44800/54000]
Training loss: 2.398650  [51200/54000]
Validation loss: 2.320525  [    0/ 6000]
Avg. Validation loss: 2.360363
===EPOCH===  2
Training loss: 2.304900  [    0/54000]
Training loss: 2.351775  [ 6400/54000]
Training loss: 2.383025  [12800

# Testing

In [ ]:
def test_loop(test_loader, simple_net, criterion):
    simple_net.eval()
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            pred = simple_net(X)
            test_loss += criterion(pred, y).item()                     # testing loop for accuracy check , basically tessting the model


            pred_labels = pred.argmax(dim=1)
            y= y.argmax(dim=1)

            correct += (pred_labels == y).sum().item()

    test_loss /= num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {(100*accuracy):.2f}%, Avg loss: {test_loss:.8f} \n")

In [ ]:
test_loop(test_loader, simple_net, criterion)

Test Error: 
 Accuracy: 10.00%, Avg loss: 2.36103073 

